In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [2]:
df = pd.read_csv(r'C:\Users\Nacho Miranda Riadi\Desktop\Proyectos python\Habitaciones\Habitaciones\dataset_inquilinos.csv', index_col='id_inquilino')

In [3]:
df.columns = [
'horario', 'bioritmo', 'nivel_educativo', 'leer', 'animacion', 
'cine', 'mascotas', 'cocinar', 'deporte', 'dieta', 'fumador',
'visitas', 'orden', 'musica_tipo', 'musica_alta', 'plan_perfecto', 'instrumento'
]

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 1 to 12000
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   horario          12000 non-null  object
 1   bioritmo         12000 non-null  object
 2   nivel_educativo  12000 non-null  object
 3   leer             12000 non-null  object
 4   animacion        12000 non-null  object
 5   cine             12000 non-null  object
 6   mascotas         12000 non-null  object
 7   cocinar          12000 non-null  object
 8   deporte          12000 non-null  object
 9   dieta            12000 non-null  object
 10  fumador          12000 non-null  object
 11  visitas          12000 non-null  object
 12  orden            12000 non-null  object
 13  musica_tipo      12000 non-null  object
 14  musica_alta      12000 non-null  object
 15  plan_perfecto    12000 non-null  object
 16  instrumento      12000 non-null  object
dtypes: object(17)
memory usage: 1.6

In [6]:
encoder = OneHotEncoder(sparse_output=False)
df_encoded = encoder.fit_transform(df)

In [7]:
df_encoded.shape

(12000, 37)

In [8]:
df['musica_tipo'].value_counts()

reggaeton    3299
pop          3147
rock         2878
clasica      2676
Name: musica_tipo, dtype: int64

In [9]:
encoded_feature_names = encoder.get_feature_names_out()


In [10]:
encoded_feature_names

array(['horario_mañana', 'horario_noche', 'bioritmo_madrugador',
       'bioritmo_nocturno', 'nivel_educativo_primaria',
       'nivel_educativo_secundaria', 'nivel_educativo_universitaria',
       'leer_no', 'leer_si', 'animacion_no', 'animacion_si', 'cine_no',
       'cine_si', 'mascotas_con mascotas', 'mascotas_sin mascotas',
       'cocinar_cocinar', 'cocinar_pedir comida', 'deporte_no',
       'deporte_si', 'dieta_no', 'dieta_si', 'fumador_no', 'fumador_si',
       'visitas_no', 'visitas_si', 'orden_ordenada', 'orden_relajada',
       'musica_tipo_clasica', 'musica_tipo_pop', 'musica_tipo_reggaeton',
       'musica_tipo_rock', 'musica_alta_no', 'musica_alta_si',
       'plan_perfecto_casa', 'plan_perfecto_salir', 'instrumento_no',
       'instrumento_si'], dtype=object)

In [11]:
# 4. MATRIZ DE SIMILIARIDAD
# Calcular la matriz de similaridad utilizando el punto producto
matriz_s = np.dot(df_encoded, df_encoded.T)

# Define el rango de destino
rango_min = -100
rango_max = 100

# Encontrar el mínimo y máximo valor en matriz_s
min_original = np.min(matriz_s)
max_original = np.max(matriz_s)


In [12]:
matriz_s_reescalada = ((matriz_s - min_original) / (max_original - min_original)) * (rango_max - rango_min) + rango_min


In [13]:
# Pasar a Pandas
df_similaridad = pd.DataFrame(matriz_s_reescalada,
        index = df.index,
        columns = df.index)


# 5. BÚSQUEDA DE INQUILINOS COMPATIBLES

In [14]:
def inquilinos_compatibles(id_inquilinos, topn):
    # Verificar si todos los ID de inquilinos existen en la matriz de similaridad
    for id_inquilino in id_inquilinos:
        if id_inquilino not in df_similaridad.index:
            return 'Al menos uno de los inquilinos no encontrado'

    # Obtener las filas correspondientes a los inquilinos dados
    filas_inquilinos = df_similaridad.loc[id_inquilinos]

    # Calcular la similitud promedio entre los inquilinos
    similitud_promedio = filas_inquilinos.mean(axis=0)

    # Ordenar los inquilinos en función de su similitud promedio
    inquilinos_similares = similitud_promedio.sort_values(ascending=False)

    # Excluir los inquilinos de referencia (los que están en la lista)
    inquilinos_similares = inquilinos_similares.drop(id_inquilinos)

    # Tomar los topn inquilinos más similares
    topn_inquilinos = inquilinos_similares.head(topn)

    # Obtener los registros de los inquilinos similares
    registros_similares = df.loc[topn_inquilinos.index]

    # Obtener los registros de los inquilinos buscados
    registros_buscados = df.loc[id_inquilinos]

    # Concatenar los registros buscados con los registros similares en las columnas
    resultado = pd.concat([registros_buscados.T, registros_similares.T], axis=1)

    # Crear un objeto Series con la similitud de los inquilinos similares encontrados
    similitud_series = pd.Series(data=topn_inquilinos.values, index=topn_inquilinos.index, name='Similitud')

    # Devolver el resultado y el objeto Series
    return(resultado, similitud_series)

In [21]:
id_inquilinos = [1]
topn = 1
resultado, similitud_series = inquilinos_compatibles(id_inquilinos, topn)

# Mostrar los resultados
print("Resultados de inquilinos compatibles:")
print(resultado)
print("\nSimilitud de inquilinos compatibles:")
print(similitud_series)

Resultados de inquilinos compatibles:
id_inquilino             1             6710
horario                 noche         noche
bioritmo             nocturno      nocturno
nivel_educativo    secundaria    secundaria
leer                       si            si
animacion                  no            si
cine                       no            no
mascotas         con mascotas  con mascotas
cocinar          pedir comida  pedir comida
deporte                    si            si
dieta                      si            si
fumador                    si            si
visitas                    si            si
orden                relajada      relajada
musica_tipo         reggaeton     reggaeton
musica_alta                no            no
plan_perfecto            casa          casa
instrumento                si            si

Similitud de inquilinos compatibles:
id_inquilino
6710    88.235294
Name: Similitud, dtype: float64
